In [3]:
import pandas as pd
import numpy as np

In [107]:
from lightfm import LightFM

C:\Users\taeny\AppData\Local\Continuum\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


Import Data
---

In [49]:
#import food rating data
df_food = pd.read_csv('data\\food_rating.csv')
df_food_rating.head()

In [52]:
df_food2 = df_food.reset_index()
df_food2 = df_food2.rename(columns = {'index':'userid'})
df_food2.head()

Divide Data into User info., Preference, and Rating
--

In [93]:
user_info_index = [0,1,2,3,4]  + list(range(54,58))
food_drop_index = list(range(0,5)) + list(range(54,58))

In [94]:
df_user_info = df_food2.iloc[:,user_info_index]
df_food_rating = df_food2.drop(axis =1,columns=df_food2.columns[food_drop_index])
df_food_rating.head()

In [96]:
df_food_rating.head()

,찜 (갈비찜/찜닭 등),갈비탕/설렁탕,곱창/막창,볶음밥,김치찌개,된장찌개,닭갈비,닭도리탕,불고기,냉면(물/비빔),...,카레/커리,김밥,분식(떡볶이/튀김/순대),라면,오므라이스,컵밥,브리또&타코,햄버거,샌드위치,치킨
0,2,4,4,3,2,4,3,1,3,4,...,3,3,3,4,3,2,4,5,5,4
1,5,5,0,5,5,5,2,2,5,2,...,3,3,1,1,5,4,5,1,1,3
2,4,4,3,4,3,5,3,2,4,3,...,3,2,4,4,4,3,2,4,3,4
3,4,4,5,4,4,2,4,5,4,4,...,3,3,5,3,4,2,3,3,4,4
4,3,5,3,5,4,4,5,3,5,5,...,5,5,4,5,5,4,5,5,5,5


In [81]:
df_food_rating_stack = pd.DataFrame(df_food_rating.stack()).reset_index() 
df_food_rating_stack.head()

In [87]:
df_food_rating_stack.columns =['userid','food','rating']

In [91]:
df_food_rating_stack.head()


,userid,food,rating
0,0,찜 (갈비찜/찜닭 등),2
1,0,갈비탕/설렁탕,4
2,0,곱창/막창,4
3,0,볶음밥,3
4,0,김치찌개,2


In [92]:
df_food_rating_stack.shape

(7791, 3)

In [104]:
df_food_rating_stack.head()

,userid,food,rating
0,0,찜 (갈비찜/찜닭 등),2
1,0,갈비탕/설렁탕,4
2,0,곱창/막창,4
3,0,볶음밥,3
4,0,김치찌개,2


In [105]:
df_food_rating_stack_nonzero = df_food_rating_stack[df_food_rating_stack['rating'] != 0]

In [109]:
from sklearn.utils import shuffle

In [116]:
df_food_rating_nonzero = shuffle(df_food_rating_stack_nonzero)
df_food_rating_nonzero = df_food_rating_nonzero.reset_index(drop=True)

In [117]:
df_food_rating_nonzero.head()

,userid,food,rating
0,93,콩국수,1
1,3,닭도리탕,5
2,64,돈부리(일본식 덮밥),3
3,114,피자,5
4,157,찜 (갈비찜/찜닭 등),4


In [149]:
df_food_grouped = df_food_rating_nonzero.groupby(['food']).agg({'rating': 'count'}).reset_index()
df_food_grouped.head()

,food,rating
0,갈비탕/설렁탕,157
1,게장,146
2,곱창/막창,151
3,김밥,159
4,김치찌개,159


Popularity Based Recommneder
==

In [178]:
df_food_grouped = df_food_rating_nonzero.groupby(['food']).agg({'rating': 'sum'}).reset_index()
grouped_rating = df_food_rating_nonzero.groupby(['food']).agg({'userid': 'count'}).reset_index()
df_food_grouped['rating']  = df_food_grouped['rating'].div(grouped_rating['userid'])
df_food_ranking = df_food_grouped.sort_values(['rating', 'food'], ascending = [0,1])

Most Favored 10 Foods
--

In [179]:
df_food_ranking.head(10)

,food,rating
37,초밥,4.388535
21,삼겹살(구이),4.379747
27,스테이크,4.360759
38,치킨,4.295597
43,파스타,4.194969
25,수육/보쌈,4.189873
48,회(사시미),4.167742
44,피자,4.037736
17,분식(떡볶이/튀김/순대),4.006289
2,곱창/막창,4.000000


Least Favored 10 Foods
--

In [180]:
df_food_ranking.tail(10)

,food,rating
15,베트남 쌀국수,3.512658
22,삼계탕,3.496855
3,김밥,3.459119
23,샌드위치,3.383648
20,비빔밥,3.360759
26,수제비,3.327044
30,월남쌈,3.322581
45,함박스테이크,3.312102
41,컵밥,3.093960
42,콩국수,2.743243


Food Recommender
==

In [190]:
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id, rating):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
        self.rating = rating
        
        #df_food_grouped = df_food_rating_nonzero.groupby(['food']).agg({'rating': 'sum'}).reset_index()
        #grouped_rating = df_food_rating_nonzero.groupby(['food']).agg({'rating': 'count'}).reset_index()
        #df_food_grouped['rating']  = df_food_grouped['rating'].div(grouped_rating['rating'])
        #df_food_ranking = df_food_grouped.sort_values(['rating', 'food'], ascending = [0,1])

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.rating: 'sum'}).reset_index()
        grouped_rating = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped[self.rating]  = train_data_grouped[self.rating].div(grouped_rating[self.user_id])
        train_data_grouped.rename(columns = {'rating': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations

In [171]:
from sklearn.model_selection import train_test_split
df_food_train,df_food_test = train_test_split(df_food_rating_nonzero,test_size = 0.2)

In [176]:
df_food_train.head()

,userid,food,rating
6835,45,샤브샤브,3
4206,152,분식(떡볶이/튀김/순대),5
1580,113,돈부리(일본식 덮밥),5
4020,87,찜 (갈비찜/찜닭 등),5
752,71,김밥,4


In [191]:
pm = popularity_recommender_py()
pm.create(df_food_train,'userid','food','rating')

In [196]:
user_id = 
pm.recommend(user_id)

,user_id,food,score,Rank
37,5,초밥,4.390244,1.0
21,5,삼겹살(구이),4.328125,2.0
27,5,스테이크,4.312500,3.0
38,5,치킨,4.285714,4.0
25,5,수육/보쌈,4.220339,5.0
43,5,파스타,4.214815,6.0
48,5,회(사시미),4.098485,7.0
17,5,분식(떡볶이/튀김/순대),4.070312,8.0
2,5,곱창/막창,4.008130,9.0
7,5,닭갈비,4.000000,10.0
